In [5]:
import pandas as pd

ts_data=pd.read_parquet(r"C:\Users\karthikeya\taxi_demand_predictor\data\transformed\ts_data_2022_01.parquet")

ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,5,4
1,2022-01-01 01:00:00,14,4
2,2022-01-01 02:00:00,29,4
3,2022-01-01 03:00:00,8,4
4,2022-01-01 04:00:00,10,4
...,...,...,...
191203,2022-01-31 19:00:00,0,176
191204,2022-01-31 20:00:00,0,176
191205,2022-01-31 21:00:00,0,176
191206,2022-01-31 22:00:00,0,176


In [3]:
ts_data_one_location = ts_data.loc[ts_data.pickup_location_id==43, :].reset_index(drop=True)
ts_data_one_location.head(24)

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,87,43
1,2022-01-01 01:00:00,60,43
2,2022-01-01 02:00:00,32,43
3,2022-01-01 03:00:00,8,43
4,2022-01-01 04:00:00,6,43
5,2022-01-01 05:00:00,4,43
6,2022-01-01 06:00:00,4,43
7,2022-01-01 07:00:00,9,43
8,2022-01-01 08:00:00,8,43
9,2022-01-01 09:00:00,17,43


In [8]:
def get_cut_off_indices(
        data : pd.DataFrame,
        n_features : int,
        step_size : int
    ) -> list:

    stop_position = len(data)-1

    #Start the first sub-sequence at a index position 0

    subseq_first_idx = 0
    subseq_mid__idx = n_features 
    subseq_last_idx = n_features + 1
    indices = []
    
    while subseq_last_idx <= stop_position :
        indices.append((subseq_first_idx, subseq_mid__idx, subseq_last_idx))

        subseq_first_idx += step_size
        subseq_mid__idx += step_size
        subseq_last_idx += step_size

    return indices



In [5]:
n_features = 24
step_size = 1
indices = get_cut_off_indices(
    ts_data_one_location,
    n_features,
    step_size
)

In [6]:
ts_data_one_location.iloc[0:2]

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,87,43
1,2022-01-01 01:00:00,60,43


In [7]:
indices[0][1:2]

(24,)

In [21]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

NameError: name 'indices' is not defined

In [9]:
ts_data_one_location.iloc[1:2]

,pickup_hour,rides,pickup_location_id
1,2022-01-01 01:00:00,60,43


In [10]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 87.,  60.,  32., ...,  22.,  16.,  10.],
       [ 60.,  32.,   8., ...,  16.,  10.,   3.],
       [ 32.,   8.,   6., ...,  10.,   3.,   2.],
       ...,
       [ 29.,  15.,  17., ..., 117.,  67.,  63.],
       [ 15.,  17.,   9., ...,  67.,  63.,  75.],
       [ 17.,   9.,   1., ...,  63.,  75.,  37.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2022-01-02 00:00:00'), Timestamp('2022-01-02 01:00:00'), Timestamp('2022-01-02 02:00:00'), Timestamp('2022-01-02 03:00:00'), Timestamp('2022-01-02 04:00:00')]


In [11]:
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]    
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,87.0,60.0,32.0,8.0,6.0,4.0,4.0,9.0,8.0,17.0,...,62.0,102.0,80.0,91.0,33.0,34.0,17.0,22.0,16.0,10.0
1,60.0,32.0,8.0,6.0,4.0,4.0,9.0,8.0,17.0,19.0,...,102.0,80.0,91.0,33.0,34.0,17.0,22.0,16.0,10.0,3.0
2,32.0,8.0,6.0,4.0,4.0,9.0,8.0,17.0,19.0,39.0,...,80.0,91.0,33.0,34.0,17.0,22.0,16.0,10.0,3.0,2.0
3,8.0,6.0,4.0,4.0,9.0,8.0,17.0,19.0,39.0,39.0,...,91.0,33.0,34.0,17.0,22.0,16.0,10.0,3.0,2.0,1.0
4,6.0,4.0,4.0,9.0,8.0,17.0,19.0,39.0,39.0,62.0,...,33.0,34.0,17.0,22.0,16.0,10.0,3.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,64.0,37.0,29.0,15.0,17.0,9.0,1.0,1.0,1.0,2.0,...,85.0,71.0,69.0,71.0,123.0,110.0,88.0,155.0,116.0,117.0
715,37.0,29.0,15.0,17.0,9.0,1.0,1.0,1.0,2.0,1.0,...,71.0,69.0,71.0,123.0,110.0,88.0,155.0,116.0,117.0,67.0
716,29.0,15.0,17.0,9.0,1.0,1.0,1.0,2.0,1.0,3.0,...,69.0,71.0,123.0,110.0,88.0,155.0,116.0,117.0,67.0,63.0
717,15.0,17.0,9.0,1.0,1.0,1.0,2.0,1.0,3.0,10.0,...,71.0,123.0,110.0,88.0,155.0,116.0,117.0,67.0,63.0,75.0


In [26]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
        ts_data: pd.DataFrame,
        input_seq_len: int,
        step_size: int
) -> pd.DataFrame :
    """
    Slices and transposes data from time-series format into a (features, target)
    format that we can use to train supervised ML models
    """
    assert set(ts_data.columns)== {'pickup_hour', 'rides', 'pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        # keep only ts data for this `location_id`
        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id==location_id,
            ['pickup_hour', 'rides']
        ]

        # pre-compute cutoff indices to split dataframe rows

        indices = get_cut_off_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        # slice and transpose data into numpy arrays for features and targets
        n_examples= len(indices)
        x = np.ndarray(shape= (n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []

        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # NUMPY -->> PANDAS
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # NUMPY -->> PANDAS
        targets_one_location = pd.DataFrame(y, columns =[f"target_rides_next_hour"])

        # concatenate results

        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']



In [27]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7*1, # one week of history
    step_size=24,
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

  0%|          | 0/257 [00:00<?, ?it/s]C:\Users\karthikeya\AppData\Local\Temp\ipykernel_14516\2846818175.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
C:\Users\karthikeya\AppData\Local\Temp\ipykernel_14516\2846818175.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
C:\Users\karthikeya\AppData\Local\Temp\ipykernel_14516\2846818175.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before

features.shape=(6168, 170)
targets.shape=(6168,)
